In [1]:
#Import dependencies
import os
import pandas as pd
import time
from sqlalchemy import create_engine

In [2]:
#Name files
ExtractStart = time.time()
titlesFile = os.path.join("Resources","netflix_titles.csv")
moviesFile = os.path.join("Resources","movies_all_streaming.csv")
tvshowsFile = os.path.join("Resources","tv_shows_all_streaming.csv")

In [3]:
titles_df = pd.read_csv(titlesFile)

movies_df = pd.read_csv(moviesFile)

tvshows_df = pd.read_csv(tvshowsFile)

ExtractEnd = time.time()
ExtractTime = ExtractEnd-ExtractStart
TransformStart = ExtractEnd

In [4]:
titlesClean = titles_df[
                        ["type","title","country","date_added","release_year","rating"]]
titlesClean = titlesClean.rename(columns={
    'release_year':'year'
})
titlesClean['date_added'] = pd.to_datetime(titlesClean['date_added'])

In [18]:
moviesClean = movies_df[
    ["Title","Year","Age","IMDb","Rotten Tomatoes","Netflix","Country"]
]

mC_all= len(moviesClean)
moviesClean = moviesClean.loc[moviesClean['Netflix']==1]
mC_Netflix = len(moviesClean)
moviesClean = moviesClean.drop(columns='Netflix')
moviesClean = moviesClean.rename(columns={
    'Title':'title',
    'Year':'year',
    'Age':'age',
    'IMDb':'imdb',
    'Rotten Tomatoes':"rotten_tomatoes",
    'Country':'country'})

In [6]:
movieMerge = pd.merge(titlesClean, moviesClean, on=['title','year'], how='inner')
mM_all = len(movieMerge)
movieMerge = movieMerge.dropna()
mM_complete = len(movieMerge)

movieMerge = movieMerge.rename(columns={'country_x':"country"})
movieMerge = movieMerge.drop(columns='country_y')
movieMerge.reset_index(inplace=True,drop=True)
mLoss = mC_all - mM_complete

In [7]:
tvshowsClean = tvshows_df[["Title","Year","Age","IMDb","Rotten Tomatoes","Netflix"]]

tvC_all= len(tvshowsClean)

tvshowsClean = tvshowsClean.loc[tvshowsClean['Netflix']==1]
tvC_Netflix = len(tvshowsClean)
tvshowsClean = tvshowsClean.drop(columns='Netflix')
tvshowsClean = tvshowsClean.rename(columns={
    'Title':'title',
    'Year':'year',
    'Age':'age',
    'IMDb':'imdb',
    'Rotten Tomatoes':"rotten_tomatoes"
    })

In [8]:
tvshowsMerge = pd.merge(titlesClean, tvshowsClean, on=['title','year'], how='inner')
tvM_all = len(tvshowsMerge)
tvshowsMerge = tvshowsMerge.dropna()
tvM_complete = len(tvshowsMerge)

tvshowsMerge.reset_index(inplace=True,drop=True)
tvLoss = tvC_all - tvM_complete

TransformEnd = time.time()
TransformTime = TransformEnd-TransformStart
LoadStart = TransformEnd

In [9]:
connection_string = "postgres:postgres@localhost:5432/AreYouStillWatching_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
engine.table_names()

['final']

In [11]:
movieMerge.to_sql(name='final', con=engine, if_exists='append', index=False)
tvshowsMerge.to_sql(name='final', con=engine, if_exists='append', index=False)

In [19]:
final_df = pd.read_sql(sql='final',con=engine)
final_df.to_csv("Output/finalNetflix.csv")
final_len = len(final_df)

In [13]:
LoadEnd = time.time()
LoadTime = LoadEnd-LoadStart

In [16]:
ETLAudit = pd.DataFrame(
    {'domain' : 'final',
    'row_counts' : final_len,
    'extraction_time':str(round(ExtractTime,2)) + ' seconds',
    'transform_time':str(round(TransformTime,2)) + ' seconds',
    'load_time':str(round(LoadTime,2)) + ' seconds',
    'movie_errors':mLoss,
    'tv_errors':tvLoss
    },index=[0])
ETLAudit.to_html('Output/Audit.html')